In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import json
import mysql.connector
from IPython.display import clear_output

In [17]:
df = pd.read_csv("Amazon Scraping - Sheet1.csv", index_col="Unnamed: 0")

base_url = "https://www.amazon.%s/dp/%s"

time_at_100 = []

bad_index = set()

storage = dict()

mydb = mysql.connector.connect(
  host="localhost",
  user="temp",
  password="temp123",
  database="CREDICXO"
)

mycursor = mydb.cursor()
sql = "INSERT INTO data (id, title, img_url, price, details) VALUES (%s, %s, %s, %s, %s)"

In [18]:
# def is_book(soup):
#     t = soup.find("a", class_="a-link-normal a-color-tertiary").get_text().strip()
#     return (t == "Books") or (t == "Libros") or (t == "Livres") or (t == "Libri")


In [19]:
def get_title(soup):
    return soup.find("span", id="productTitle").get_text().strip()



In [20]:
def get_img_url(soup):
    try:
        img_url = soup.find("div" ,id="main-image-container").find("img").get("src").strip()
    except:
        try:
            img_url = soup.find("img", id="imgBlkFront").get("src")
        except:
            img_url = soup.find("img", id="landingImage").get("src").strip()
    return img_url

In [21]:
def get_price(soup):
    try:
        try:
            price = soup.find("span" ,class_="a-size-base a-color-price a-color-price").get_text().strip()
        except:
            price = soup.find_all("span", class_="a-color-base")[-1].get_text().strip()

    except:
        price = soup.find_all("span", class_="a-size-base a-color-price")[-1].get_text().strip()

    return price

In [22]:
def get_details(soup):
    try:
        dtls_list = soup.find_all("div",id="detailBullets_feature_div")[0]
        dtls_list = dtls_list.find("ul",class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")
        dtls_list = dtls_list.find_all("span", class_="a-list-item")

        details = dict()

        for dtl in dtls_list:
            k,v = dtl.find_all("span")
            details[k.get_text().replace("\u200f","").replace("\u200e","").replace(":",'').strip()] = v.get_text().strip()
            

    except:
        table = soup.find("table" ,id="productDetails_techSpec_section_1")
        keys = [th.get_text().strip() for th in table.find_all("th")]
        values = [td.get_text().strip().replace("\u200e","") for td in table.find_all("td")]
        details = dict(zip(keys,values))

    return details

In [23]:
def get_soup(i):

    x = tuple(df.loc[i][["country", "Asin"]])
    url = base_url % (x)
    result = requests.get(url, headers=headers)

    assert result.status_code == 200, f"{url} not available"

    data = result.text
    soup = BeautifulSoup(data, "html.parser")

    return soup.body

In [24]:
def store_data(i, tries=0):
    try:
        soup = get_soup(i)

        title = get_title(soup)
        img_url = get_img_url(soup)
        price = get_price(soup)
        details = get_details(soup)

        storage[str(df.loc[i]["id"])] = {
            "title": title,
            "image_url": img_url,
            "price": price,
            "details": details,
        }

        val = (int(df.loc[i]['id']), title, img_url, price, str(details))
        mycursor.execute(sql, val)


    except AssertionError as err:
        print(err)

    except Exception as exp:
        print("Error at :", i)
        bad_index.add(i)
        print(exp)
        if tries < 2:
            print("tried :", tries + 2)
            store_data(i, tries + 1)
        else:
            print("Tried maximum times")

In [28]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:101.0) Gecko/20100101 Firefox/101.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Origin': 'https://www.amazon.fr',
    'Connection': 'keep-alive',
    'Referer': 'https://www.amazon.fr/',
    'Sec-Fetch-Dest': 'script',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
}

In [29]:
start = time.time()

for i in range(700,1000):

    store_data(i)

    if i % 100 == 0:
        tt = time.time() - start
        time_at_100.append(tt)
        start = time.time()
        print("At :", i)
        print("time took :", tt)

https://www.amazon.fr/dp/4172744 not available
At : 700
time took : 0.46044325828552246
https://www.amazon.de/dp/4172833 not available
https://www.amazon.fr/dp/4172833 not available
https://www.amazon.de/dp/4172841 not available
https://www.amazon.fr/dp/4172841 not available
https://www.amazon.de/dp/4172884 not available
https://www.amazon.fr/dp/4172884 not available
https://www.amazon.de/dp/4172906 not available
https://www.amazon.fr/dp/4172906 not available
https://www.amazon.de/dp/4172922 not available
https://www.amazon.fr/dp/4172922 not available
https://www.amazon.de/dp/4172930 not available
https://www.amazon.fr/dp/4172930 not available
https://www.amazon.de/dp/4172957 not available
https://www.amazon.fr/dp/4172957 not available
https://www.amazon.de/dp/4172973 not available
https://www.amazon.fr/dp/4172973 not available
https://www.amazon.de/dp/4173082 not available
https://www.amazon.fr/dp/4173082 not available
https://www.amazon.de/dp/4173090 not available
https://www.amazon.

In [30]:
with open("result.json", "w") as f:
    json.dump(storage, f)

In [31]:
mydb.commit()

In [32]:
for t in time_at_100:
    print(str(t))

0.44241976737976074
73.62571001052856
94.46051812171936
76.3008680343628
0.5014393329620361
68.99638438224792
80.12688302993774
0.46044325828552246
64.15927863121033
49.32078790664673


In [33]:
print("bad index :", bad_index)

bad index : set()
